# Ejemplo de `word2vec` y `fastText` con `gensim`


En la siguiente celda, importamos las librerías necesarias y configuramos los mensajes de los logs.

In [1]:
import gensim, logging, os

logging.basicConfig(
    format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO
)

## word2vec 

He entrenado un modelo de word2vec con la Wikipedia en español, [tal y como explico aquí](https://github.com/vitojph/kschool-nlp-16/tree/master/misc).

In [2]:
MODEL = "../data/eswiki-300.w2v"
model = gensim.models.Word2Vec.load(MODEL)

2020-02-08 15:07:12,105 : INFO : loading Word2Vec object from ../data/eswiki-300.w2v
2020-02-08 15:07:14,092 : INFO : loading trainables recursively from ../data/eswiki-300.w2v.trainables.* with mmap=None
2020-02-08 15:07:14,094 : INFO : loading syn1neg from ../data/eswiki-300.w2v.trainables.syn1neg.npy with mmap=None
2020-02-08 15:07:14,449 : INFO : loading vocabulary recursively from ../data/eswiki-300.w2v.vocabulary.* with mmap=None
2020-02-08 15:07:14,450 : INFO : loading wv recursively from ../data/eswiki-300.w2v.wv.* with mmap=None
2020-02-08 15:07:14,451 : INFO : loading vectors from ../data/eswiki-300.w2v.wv.vectors.npy with mmap=None
2020-02-08 15:07:14,787 : INFO : setting ignored attribute vectors_norm to None
2020-02-08 15:07:14,788 : INFO : setting ignored attribute cum_table to None
2020-02-08 15:07:14,789 : INFO : loaded ../data/eswiki-300.w2v


## Probando nuestro modelo

El objeto `model` contiene una enorme matriz de números: una tabla, donde cada fila es uno de los términos del vocabulario reconocido y cada columna es una de las características que permiten modelar el significado de dicho término.

En nuestro modelo, tal y como está entrenado, tenemos más de 41 millones de términos:

In [3]:
print(model.corpus_count)

41843901


Cada término del vocabulario está representado como un vector con 300 dimensiones. Podemos acceder al vector de un término concreto:

In [4]:
print(model.wv["azul"])
print(model.wv["verde"])
print(model.wv["clorofila"])

[ 6.97230875e-01 -2.16513014e+00  3.48829389e-01 -9.89584625e-01
 -1.02388668e+00 -2.17564240e-01 -1.96011472e+00  2.94649076e+00
 -9.12335694e-01 -6.84771001e-01  1.52577829e+00  7.75085762e-02
 -7.02408105e-02  4.89242375e-01  1.37050912e-01 -9.30723965e-01
  6.80433512e-02 -1.08530678e-01  2.31580353e+00 -5.53485096e-01
 -3.57593626e-01 -2.28935838e+00 -1.50510311e-01  1.26760174e-02
  6.07259810e-01 -2.17627048e+00  2.73906016e+00  1.09757566e+00
 -1.71337926e+00  1.64041713e-01  6.90737784e-01  1.26919913e+00
 -5.12029469e-01 -2.01885894e-01  1.78642869e-01  1.53002536e+00
 -1.43445313e-01  1.11213720e+00 -1.41976345e+00 -1.72563398e+00
  7.15158105e-01  1.11852288e+00 -5.18750012e-01 -7.85638809e-01
 -2.75143218e+00 -2.21702838e+00 -7.32014835e-01 -4.00392205e-01
  1.58708438e-01  1.02694380e+00 -2.80862898e-01  1.41663051e+00
  1.11722755e+00 -1.24235370e-03 -1.14149344e+00 -1.97176194e+00
 -2.22860837e+00 -1.23289347e+00  1.74586341e-01  4.37671602e-01
 -9.38071907e-01  4.78671

Estos vectores no nos dicen mucho, salvo que contienen números muy pequeños :-/

El mismo objeto `model` permite acceder a una serie de funcionalidades ya implementadas que nos van a permitir evaluar formal e informalmente el modelo. Por el momento, nos contentamos con los segundo: vamos a revisar visualmente los significados que nuestro modelo ha aprendido por su cuenta. 

Podemos calcular la similitud semántica entre dos términos usando el método `similarity`, que nos devuelve un número entre 0 y 1:

In [5]:
print("hombre - mujer", model.wv.similarity("hombre", "mujer"))

print("perro - gato", model.wv.similarity("perro", "gato"))

print("gato - periódico", model.wv.similarity("gato", "periódico"))

print("febrero - azul", model.wv.similarity("febrero", "azul"))

hombre - mujer 0.43087164
perro - gato 0.8005251
gato - periódico 0.17121994
febrero - azul -0.021135561


Podemos seleccionar el término que no encaja a partir de una determinada lista de términos usando el método `doesnt_match`:

In [6]:
lista1 = "madrid barcelona gonzález washington".split()
print(f"""en la lista '{" ".join(lista1)}' sobra '{model.wv.doesnt_match(lista1)}'""")

lista2 = "psoe pp ciu ronaldo".split()
print(f"""en la lista '{" ".join(lista2)}' sobra '{model.wv.doesnt_match(lista2)}'""")

lista3 = "publicaron declararon soy fueron negaron".split()
print(f"""en la lista '{" ".join(lista3)}' sobra '{model.wv.doesnt_match(lista3)}'""")

lista4 = "homero saturno cervantes shakespeare cela".split()
print(f"""en la lista '{" ".join(lista4)}' sobra '{model.wv.doesnt_match(lista4)}'""")

lista5 = "madrid barcelona alpedrete marsella".split()
print(f"""en la lista '{" ".join(lista5)}' sobra '{model.wv.doesnt_match(lista5)}'""")

2020-02-08 15:07:16,601 : INFO : precomputing L2-norms of word weight vectors


en la lista 'madrid barcelona gonzález washington' sobra 'washington'
en la lista 'psoe pp ciu ronaldo' sobra 'ronaldo'
en la lista 'publicaron declararon soy fueron negaron' sobra 'soy'
en la lista 'homero saturno cervantes shakespeare cela' sobra 'saturno'
en la lista 'madrid barcelona alpedrete marsella' sobra 'alpedrete'


/home/victor/.conda/envs/nlp/lib/python3.6/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


Podemos buscar los términos más similares usando el método `most_similar` de nuestro modelo:

In [7]:
terminos = "azul madrid bmw bici 2019 rock google shakira jay-z xiaomi rajoy brexit saturno césar lazio".split()

for t in terminos:
    print(f"{t} ==> {model.wv.most_similar(t)}\n")

azul ==> [('verde', 0.7170504331588745), ('amarillo', 0.7170424461364746), ('rojo', 0.7002288103103638), ('azúl', 0.6983712315559387), ('turquesa', 0.6890928745269775), ('morado', 0.6765597462654114), ('carmesí', 0.6581361889839172), ('granate', 0.6548363566398621), ('naranja', 0.6507274508476257), ('beige', 0.6397430896759033)]

madrid ==> [('valladolid', 0.7847391366958618), ('barcelona', 0.775865912437439), ('sevilla', 0.760832667350769), ('zaragoza', 0.7517766952514648), ('madrid.', 0.7219818830490112), ('valencia', 0.6873288154602051), ('oviedo', 0.6831979751586914), ('salamanca', 0.6781078577041626), ('málaga', 0.675739049911499), ('pamplona', 0.6748294830322266)]

bmw ==> [('audi', 0.7591230869293213), ('porsche', 0.7583965063095093), ('mercedes-benz', 0.7572087645530701), ('renault', 0.7485953569412231), ('toyota', 0.7476786375045776), ('opel', 0.739383339881897), ('volkswagen', 0.7309398055076599), ('maserati', 0.7237600088119507), ('gt', 0.7185136675834656), ('volvo', 0.71532

Con el mismo método `most_similar` podemos combinar vectores de palabras tratando de jugar con los rasgos semánticos de cada una de ellas para descubrir nuevas relaciones.

In [8]:
print("mujer que ejerce la autoridad en una alcaldía ==> alcalde + mujer - hombre")
most_similar = model.wv.most_similar(
    positive=["alcalde", "mujer"], negative=["hombre"], topn=3
)
for item in most_similar:
    print(item)

print(
    "mujer especializada en alguna terapia de la medicina ==> doctor + mujer - hombre"
)
most_similar = model.wv.most_similar(
    positive=["doctor", "mujer"], negative=["hombre"], topn=3
)
for item in most_similar:
    print(item)

print("monarca soberano ==> reina + hombre - mujer")
most_similar = model.wv.most_similar(
    positive=["reina", "hombre"], negative=["mujer"], topn=3
)
for item in most_similar:
    print(item)

print("capital de Alemania ==> moscú + alemania - rusia")
most_similar = model.wv.most_similar(
    positive=["moscú", "alemania"], negative=["rusia"], topn=3
)
for item in most_similar:
    print(item)

print("presidente de Francia ==> obama + francia - eeuu")
most_similar = model.wv.most_similar(
    positive=["obama", "francia"], negative=["eeuu"], topn=3
)
for item in most_similar:
    print(item)

mujer que ejerce la autoridad en una alcaldía ==> alcalde + mujer - hombre
('alcaldesa', 0.7252962589263916)
('regidora', 0.6073892116546631)
('concejala', 0.5990710258483887)
mujer especializada en alguna terapia de la medicina ==> doctor + mujer - hombre
('doctora', 0.6626786589622498)
('enfermera', 0.5988627672195435)
('esposa', 0.5201101303100586)
monarca soberano ==> reina + hombre - mujer
('rey', 0.6301776170730591)
('monarca', 0.5567039251327515)
('príncipe', 0.5296465158462524)
capital de Alemania ==> moscú + alemania - rusia
('berlín', 0.7966121435165405)
('múnich', 0.7589867115020752)
('hamburgo', 0.7321966886520386)
presidente de Francia ==> obama + francia - eeuu
('hollande', 0.4829712212085724)
('papado', 0.45290911197662354)
('napoleón', 0.4527309238910675)


## fastText

In [10]:
from gensim.models.keyedvectors import Word2VecKeyedVectors

MODEL = "../data/cc.en.300.vec"
fasttext = Word2VecKeyedVectors.load_word2vec_format(MODEL)

2020-02-08 15:10:07,207 : INFO : loading projection weights from ../data/cc.en.300.vec
2020-02-08 15:18:04,090 : INFO : loaded (2000000, 300) matrix from ../data/cc.en.300.vec


In [11]:
print(len(fasttext.vocab))

2000000


In [12]:
print(fasttext["google"])
print(fasttext["Google"])

[-0.0271 -0.0098  0.0774  0.0583  0.0299 -0.0128  0.0549  0.02    0.0832
 -0.0304  0.0518 -0.0156  0.1136  0.0578 -0.0315  0.0225  0.0616 -0.0601
  0.0368 -0.002   0.0005 -0.0073  0.0436  0.0322  0.0299  0.0709 -0.0234
  0.095  -0.0121  0.1088  0.0547 -0.0087 -0.0978 -0.0141  0.0333 -0.0488
 -0.0365  0.0512  0.0976  0.0157  0.0037 -0.0768 -0.0619 -0.023  -0.001
  0.083  -0.0548  0.0709 -0.0049  0.0537 -0.0042 -0.0442  0.0676  0.0115
  0.0061 -0.0051 -0.0135  0.0432 -0.0854 -0.0006 -0.038  -0.0077 -0.0222
  0.014  -0.0092  0.1031 -0.0114 -0.0827 -0.0452  0.0223 -0.0192 -0.0613
 -0.0724  0.0005 -0.0893 -0.0369  0.0982  0.0778 -0.0409 -0.0266 -0.0198
  0.0172 -0.0789  0.0652  0.0136  0.0716  0.0195  0.0466  0.087  -0.0316
 -0.054   0.0148  0.081   0.0166  0.0424 -0.0067  0.0104  0.1335 -0.0011
 -0.0684  0.0147  0.0002 -0.0033  0.0682  0.0658 -0.046  -0.0615  0.0014
  0.0343 -0.0132 -0.0083  0.0179  0.0359 -0.0129 -0.0319 -0.0939 -0.0418
  0.0137  0.0129  0.0417 -0.0201  0.0922  0.0622  0.

In [13]:
print(fasttext.similarity("google", "Google"))
print(fasttext.similarity("google", "search"))

0.66849655
0.57657176


In [14]:
terms = "Sun chess boy president hdmi England French Google google keyboard mouse plant".split()

for t in terms:
    print(f"{t} ==> {fasttext.most_similar(t)}\n")

2020-02-08 15:19:20,145 : INFO : precomputing L2-norms of word weight vectors


Sun ==> [('Sun.The', 0.6877390146255493), ('Sun.I', 0.6652367115020752), ('SUn', 0.6346933841705322), ('Sat', 0.6030572652816772), ('-Sun', 0.6014525890350342), ('.Sun', 0.5839200019836426), ('Mon', 0.5815243721008301), ('Sun.', 0.5772802233695984), ('Fri', 0.5570669174194336), ('Sun4', 0.5426012873649597)]

chess ==> [('Chess', 0.7488962411880493), ('non-chess', 0.7266454100608826), ('backgammon', 0.7085219025611877), ('chessplayers', 0.7080339193344116), ('chess.', 0.7011615037918091), ('chess-playing', 0.6884827613830566), ('chessboard', 0.6804870367050171), ('chess-related', 0.6718202829360962), ('chessmaster', 0.6695749759674072), ('chessplayer', 0.6695084571838379)]

boy ==> [('girl', 0.8075398206710815), ('boy.He', 0.7046689987182617), ('boy-', 0.6979820728302002), ('boy.It', 0.6886506080627441), ('boys', 0.6847472190856934), ('boy.But', 0.6818395853042603), ('boy.This', 0.6790170669555664), ('boy.The', 0.6763201355934143), ('kid', 0.6720180511474609), ('boy.', 0.669898629188537

## Ejercicio: Crea embeddings de oraciones y calcula similitud semántica

1. Procesamos una oración completa
2. Tokenizamos la frase
3. Sustituimos cada token por su vector
4. Calculamos el promedio de los vectores

In [29]:
import numpy as np
from nltk import word_tokenize

def embed(text: str) -> np.array:
    """Crea el embedding de un documento combinando los vectores de palabras de fastText"""
    tokens = word_tokenize(text)
    word_embeddings = [fasttext[token] for token in tokens]
    doc_embedding = np.mean(word_embeddings, axis=0) 
    return doc_embedding

In [30]:
embedding = embed("This is just an example.")
print(type(embedding))
print(embedding.shape)

<class 'numpy.ndarray'>
(300,)


In [31]:
texts = [
    "Pizza and pasta are my favorite food!", 
    "I hate football", 
    "I have a blue Toyota Corolla.", 
    "I enjoy playing tennis.",
    "I love ice-cream :-)",
    "Green colorless ideas sleep furiously.",
    "Relations between Washington and Beijing have been tense for years.",
    "Chinese officials have criticized the United States.",
    "U.S. Women's Team Qualifies for Olympic Soccer Tournament"
]

doc_embeddings = [embed(text) for text in texts]

In [32]:
def knn_search(text: str, k: int = 3) -> None:
    """Búsqueda basada en vecinos cercanos (KNN)"""
    text_vec = embed(text)
    score = np.sum(text_vec * doc_embeddings, axis=1) / np.linalg.norm(doc_embeddings, axis=1)
    topk_idx = np.argsort(score)[::-1][:k]
    for idx in topk_idx:
        print(f"{texts[idx]} -> {score[idx]}")

In [28]:
knn_search("I own a reliable Japanese car")

I have a blue Toyota Corolla. -> 1.0283704996109009
I enjoy playing tennis. -> 0.8112152814865112
I hate football -> 0.8047472834587097


In [33]:
knn_search("I enjoy sports")

I hate football -> 1.1839185953140259
I enjoy playing tennis. -> 1.1252000331878662
I have a blue Toyota Corolla. -> 0.9867241978645325


In [34]:
def cosine_similarity(text: str) -> None:
    """Calcula la similitud semántica entre un documento y una colección de documentos"""
    text_vec = embed(text)
    for i, embedding in enumerate(doc_embeddings):
        similarity = np.dot(text_vec, embedding) / (np.linalg.norm(text_vec) * np.linalg.norm(embedding))
        print(f"{texts[i]} -> {similarity}")

In [35]:
cosine_similarity("I like Italian food")

Pizza and pasta are my favorite food! -> 0.7369285821914673
I hate football -> 0.8678369522094727
I have a blue Toyota Corolla. -> 0.7520216703414917
I enjoy playing tennis. -> 0.793563187122345
I love ice-cream :-) -> 0.651222288608551
Green colorless ideas sleep furiously. -> 0.47906479239463806
Relations between Washington and Beijing have been tense for years. -> 0.5077171921730042
Chinese officials have criticized the United States. -> 0.5151601433753967
U.S. Women's Team Qualifies for Olympic Soccer Tournament -> 0.28166434168815613


In [38]:
cosine_similarity("Female sport in America")

Pizza and pasta are my favorite food! -> 0.42676427960395813
I hate football -> 0.2961423695087433
I have a blue Toyota Corolla. -> 0.4360668957233429
I enjoy playing tennis. -> 0.3851541578769684
I love ice-cream :-) -> 0.32622864842414856
Green colorless ideas sleep furiously. -> 0.30096158385276794
Relations between Washington and Beijing have been tense for years. -> 0.4897002577781677
Chinese officials have criticized the United States. -> 0.5181446075439453
U.S. Women's Team Qualifies for Olympic Soccer Tournament -> 0.5080596208572388
